<a href="https://colab.research.google.com/github/pabloac31/TFG/blob/master/cifar10_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/pabloac31/TFG.git
%cd TFG

Cloning into 'TFG'...
remote: Enumerating objects: 93, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 134 (delta 58), reused 35 (delta 16), pack-reused 41
Receiving objects: 100% (134/134), 79.08 MiB | 10.61 MiB/s, done.
Resolving deltas: 100% (64/64), done.
/content/TFG


In [2]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
from cifar10_models import *
from utils import *
from adversarial_attacks import *

In [0]:
iv3 = inception_v3(pretrained=True)

In [5]:
# Define what device we are using
use_cuda=True
print("CUDA Available: ",torch.cuda.is_available())
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

CUDA Available:  True


In [0]:
test_loader = testloader_cifar10('./data', 256)

Extracting ./data/cifar-10-python.tar.gz to ./data


In [0]:
test_model(iv3, device, test_loader)

100%|██████████| 40/40 [00:21<00:00,  1.83it/s]


0.9541

In [6]:
adv_loader = testloader_cifar10('./data', 1)

Extracting ./data/cifar-10-python.tar.gz to ./data


In [0]:
test_fgsm(iv3, device, './data/img1.png', 0.56)

In [72]:
#params = {'epsilon': 0.4}
#params = {'num_classes': 10, 'overshoot': 0.02, 'max_iter': 50, 'lambda_fac': 1.5}
params = {'lambda_': 3., 'max_iter': 20, 'epsilon': 0.2}
adv_examples = attack_model(iv3, device, adv_loader, 'sparsefool', params, iters=1)

plt.imshow(denormalize_cifar10(adv_examples[0][2]).transpose((1,2,0)))
plt.title('Original image -> ' + str(adv_examples[0][0]))
plt.show()
plt.imshow(denormalize_cifar10(adv_examples[0][3]).transpose((1,2,0)))
plt.title('Adversarial image -> ' + str(adv_examples[0][1]))
plt.show()

  0%|          | 0/10000 [00:00<?, ?it/s]


RuntimeError: ignored

In [0]:
def sparsefool(model, device, image, label, lb, ub, lambda_=3., max_iter=20, epsilon=0.02):

  x_i = copy.deepcopy(image)
  fool_im = copy.deepcopy(x_i)
  fool_label = label
  
  loops = 0

  while fool_label == label and loops < max_iter:

    normal, x_adv = deepfool(model, device, x_i, lambda_fac=lambda_)

    x_i = linear_solver(x_i, normal, x_adv, lb, ub)

    fool_im = x_0 + (1 + epsilon) * (x_i - x_0)
    fool_im = clip_image_values(fool_im, lb, ub)
    fool_label = torch.argmax(model.forward(Variable(fool_im, requires_grad=True)).data).item()

    loops += 1

  r = fool_im - x_0
  return fool_im

In [0]:
def clip_image_values(x, minv, maxv):

    x = torch.max(x, minv)
    x = torch.min(x, maxv)
    return x

In [0]:
def linear_solver(x_0, normal, boundary_point, lb, ub):

  input_shape = x_0.size()

  coord_vec = copy.deepcopy(normal)
  plane_normal = copy.deepcopy(coord_vec).view(-1)
  plane_point = copy.deepcopy(boundary_point).view(-1)

  x_i = copy.deepcopy(x_0)

  f_k = torch.dot(plane_normal, x_0.view(-1) - plane_point)
  sign_true = f_k.sign().item()

  beta = 0.001 * sign_true
  current_sign = sign_true

  while current_sign == sign_true and coord_vec.nonzero().size()[0] > 0:

    f_k = torch.dot(plane_normal, x_i.view(-1) - plane_point) + beta

    pert = f_k.abs() / coord_vec.abs().max()

    mask = torch.zeros_like(coord_vec)
    mask[np.unravel_index(torch.argmax(coord_vec.abs()), input_shape)] = 1.

    r_i = torch.clamp(pert, min=1e-4) * mask * coord_vec.sign()

    x_i = x_i + r_i
    x_i = clip_image_values(x_i, lb, up)

    f_k = torch.dot(plane_normal, x_i.view(-1) - plane_point)
    current_sign = f_k.sign().item()

    coord_vec[r_i != 0] = 0

  return x_i

In [0]:
def attack_model(model, device, test_loader, method, params, iters=10000):

  # Initialize the network and set the model in evaluation mode.
  model = model.to(device).eval()

  # Stat counters
  correct = 0
  confidence = 0
  total_time = 0
  ex_robustness = 0
  model_robustness = 0
  adv_examples = []

  i = 0

  # Loop (iters) examples in test set
  for data, target in pbar(test_loader):
    if i >= iters:
      break
    i += 1

    # Send the data and label to the device
    data, target = data.to(device), target.to(device)

    # Set requires_grad attribute of tensor. Important for Attack
    if method in ['fgsm', 'deepfool', 'sparsefool']:
        data.requires_grad = True

    # Forward pass the data through the model
    output = model(data)
    init_pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability

    # If the initial prediction is wrong, dont botter attacking
    if init_pred.item() != target.item():
      continue

    if method == 'fgsm':
        # Call FGSM attack
        time_ini = time.time()
        perturbed_data = fgsm(model, data, params["epsilon"], output, target)
        time_end = time.time()
        total_time += time_end-time_ini
      
    elif method == 'deepfool':
        # Call DeepFool attack
        time_ini = time.time()
        perturbed_data = deepfool(model, device, data, params["num_classes"], params["overshoot"], params["max_iter"], params["lambda_fac"])[1]
        time_end = time.time()
        total_time += time_end-time_ini

    elif method == 'sparsefool':
        # Generate lower and upper bounds
        delta = 255
        lb, ub =  valid_bounds_cifar10(data, delta)
        lb = lb[None, :, :, :].to(device)
        ub = ub[None, :, :, :].to(device)
        perturbed_data = sparsefool(model, device, data, target.item(), lb, ub, params["lambda_"], params["max_iter"], params["epsilon"])

    # Update model robustness
    p_norm = 2
    im_np = data.squeeze().detach().cpu().numpy()
    adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
    adv_rob = np.linalg.norm((adv_ex - im_np).flatten(), ord=p_norm)
    ex_robustness += adv_rob
    model_robustness += adv_rob / np.linalg.norm(im_np.flatten(), ord=p_norm)

    # Re-classify the perturbed image
    output = model(perturbed_data)

    # Check for success
    final_pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
    confidence += F.softmax(output, dim=1).max(1, keepdim=True)[0].item()  # adv. confidence
    if final_pred.item() == target.item():
      correct += 1
    else:
      # Save some adv examples for visualization later
      if len(adv_examples) < 5:
        adv_examples.append( (init_pred.item(), final_pred.item(), im_np, adv_ex) )

  # Calculate stats
  final_acc = correct / float(iters)  # len(test_loader)
  avg_confidence = confidence / float(iters)
  avg_time = total_time / float(iters)
  avg_ex_robustness = ex_robustness / float(iters)
  model_robustness = model_robustness / float(iters)
  print("\n======== RESULTS ========")
  print("Test Accuracy = {} / {} = {}\nAverage confidence = {}\nAverage time = {}\nAverage magnitude of perturbations = {}\nModel robustness = {}"
    .format(correct, iters, final_acc, avg_confidence, avg_time, avg_ex_robustness, model_robustness))

  # Return adversarial examples
  return adv_examples

In [0]:
def deepfool(model, device, im, num_classes=10, overshoot=0.02, max_iter=50, lambda_fac=3.):

  image = copy.deepcopy(im)
  input_shape = image.size()
  
  # Array with the class probabilities of the image
  f_image = model.forward(Variable(image, requires_grad=True)).data.cpu().numpy().flatten()
  #f_image = model(image).data.cpu().numpy().flatten()
  # Classes ordered by probability (descending)
  I = f_image.argsort()[::-1]
  # We consider only 'num_classes' classes
  I = I[0:num_classes]
  # Get the predicted label
  label = I[0]

  # Start from a copy of the original image
  pert_image = copy.deepcopy(image)   # tensor of size (1,3,H,W)

  #  Initialize variables
  r_tot = torch.zeros(input_shape).to(device)
  k_i = label
  loop_i = 0

  while k_i == label and loop_i < max_iter:

    #x = pert_image.clone().detach().requires_grad_(True)
    x = Variable(pert_image, requires_grad=True)
    fs = model.forward(x)

    pert = torch.Tensor([np.inf])[0].to(device)
    w = torch.zeros(input_shape).to(device)
    
    # Calculate grad(f_label(x_i))
    fs[0, I[0]].backward(retain_graph=True)
    grad_orig = copy.deepcopy(x.grad.data)

    for k in range(1, num_classes):  # for k != label
      
      zero_gradients(x)

      # Calculate grad(f_k(x_i))
      fs[0, I[k]].backward(retain_graph=True)
      cur_grad = copy.deepcopy(x.grad.data)
      
      # Set new w_k and new f_k
      w_k = cur_grad - grad_orig
      f_k = (fs[0, I[k]] - fs[0, I[0]]).data

      # Calculate hyperplane-k distance
      pert_k = torch.abs(f_k) / w_k.norm()

      # determine which w_k to use
      if pert_k < pert:
        pert = pert_k + 0.
        w = w_k + 0.

    # compute r_i and r_tot
    # Added 1e-4 for numerical stability
    r_i = torch.clamp(pert, min=1e-4) * w / w.norm()
    r_tot = r_tot + r_i

    pert_image = pert_image + r_i
    check_fool = image + (1 + overshoot) * r_tot
    #x = check_fool.clone().detach().requires_grad_(True)
    # label assigned to x_(i+1)
    k_i = torch.argmax(model.forward(Variable(check_fool, requires_grad=True)).data).item()
    
    loop_i += 1

  #x = pert_image.clone().detach().requires_grad_(True)
  x = Variable(pert_image, requires_grad=True)
  fs = model.forward(x)
  (fs[0, k_i] - fs[0, label]).backward(retain_graph=True)
  grad = copy.deepcopy(x.grad.data)
  grad = grad / grad.norm()

  r_tot = lambda_fac * r_tot
  # (TODO) Adding clipping to maintain [0,1] range !!!!
  pert_image = image + r_tot

  return grad, pert_image

In [0]:
def valid_bounds_cifar10(img, delta=255):
  # Deepcopy of the image as a numpy int array of range [0, 255]
  im = copy.deepcopy(np.transpose(denormalize_cifar10(img.cpu().detach().numpy()[0]), (1,2,0)))
  im *= 255
  im = (np.around(im)).astype(np.int)

  # General valid bounds [0, 255]
  valid_lb = np.zeros_like(im)
  valid_ub = np.full_like(im, 255)

  # Compute the bounds
  lb = im - delta
  ub = im + delta

  # Validate that the bounds are in [0, 255]
  lb = np.maximum(valid_lb, np.minimum(lb, im))
  ub = np.minimum(valid_ub, np.maximum(ub, im))

  # Round and change types to uint8
  lb = lb.astype(np.uint8)
  ub = ub.astype(np.uint8)

  # Convert to tensors and normalize (cifar10)
  lb = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=mean_cifar10, std=std_cifar10)])(lb)
  ub = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=mean_cifar10, std=std_cifar10)])(ub)

  return lb, ub